In [1]:
from Tasks.tests import generate_test

tasks = generate_test()


In [2]:
from Tasks.GeneticAlgorithm.Permutations_generator_heuristics import generate_permutation

permutation = generate_permutation(tasks, True)


Estructura del grafo: 
{0: [Task(Name: 1), Task(Name: 2)], 1: [Task(Name: 2), Task(Name: 3), Task(Name: 4)], 2: [Task(Name: 5), Task(Name: 6)], 3: [], 4: [Task(Name: 7)], 5: [Task(Name: 7)], 6: [Task(Name: 8)], 7: [Task(Name: 9)], 8: [Task(Name: 9)], 9: []}
Orden topologico: 
[[0], [1], [2, 3, 4], [5, 6], [7, 8], [9]]
index  7
level  4
current task  7
current level  5
current task  9
current level  6
Orden topologico mutado: 
[[0], [1], [2, 3, 4], [5, 6], [8], [7], [9]]


In [3]:
print(permutation)

[Task(Name: 0), Task(Name: 1), Task(Name: 2), Task(Name: 4), Task(Name: 3), Task(Name: 5), Task(Name: 6), Task(Name: 8), Task(Name: 7), Task(Name: 9)]
